In [2]:
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import glob
import SimpleITK as sitk
import re
import pydicom
import os

In [ ]:
source_path = "/media/andres/T7 Shield/ucan_lymfom"
destination_path = os.path.join(source_path, "metadata.xlsx")

directory_list = list()
for root, dirs, files in os.walk(source_path, topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))

In [ ]:
dataset = pd.DataFrame(directory_list, columns=['directory'])
countfiles_selected = {"directory": [], "count":[]}

In [ ]:
for index, row in dataset.iterrows():
    count = 0
    for path in os.listdir(row["directory"]):
        if os.path.isfile(os.path.join(row["directory"], path)):
            count += 1
            
    countfiles_selected["directory"].append(row["directory"])
    countfiles_selected["count"].append(count)


In [ ]:
countfiles_selected_df = pd.DataFrame.from_dict(countfiles_selected)
exams_with_distorted_images_file = countfiles_selected_df[countfiles_selected_df["count"] < 179].reset_index()
print(f"Number of exams with ones file: {exams_with_distorted_images_file.shape}")
exams_with_distorted_images_file.head(2)

In [ ]:
exams_with_distorted_images_file[['source_directory', 'patient_directory', 'PET-CT_info']] = exams_with_distorted_images_file['directory'].str.rsplit(pat='/', n=2, expand=True)
exams_with_distorted_images_file["patient_directory"].nunique()

In [ ]:
exams_with_distorted_images_file.to_excel("/media/andres/T7 Shield/ucan_lymfom/Excel_files/06_11_2023/exams_with_distorted_images_file.xlsx")

In [ ]:
dataset.shape

In [ ]:
dataset = dataset[~dataset.directory.isin(exams_with_distorted_images_file.directory)]

In [ ]:
dataset.shape

In [ ]:
dataset.to_excel("/media/andres/T7 Shield/ucan_lymfom/Excel_files/06_11_2023/data_ready_for_filtering.xlsx")

In [ ]:
dataset_test = pd.read_excel("/media/andres/T7 Shield/ucan_lymfom/Excel_files/06_11_2023/data_ready_for_filtering.xlsx")

In [ ]:
type(dataset_test)

In [ ]:
dataset_test.columns

In [3]:
df = pd.read_excel("/media/andres/T7 Shield/UCAN_project/Dataframes/Generated/dataset_for_model_classification_training_new_diagnosis.xlsx")

In [4]:
df.head()

,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,unique_patient_ID_scan_date,patient_ID,scan_date,sex,diagnosis_groups,diagnosis_groups_new,GT_diagnosis_label,GT_diagnosis_label_new
0,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,npr967652562138_20100730,npr967652562138,20100730,1,C81,C81.9,0,2
1,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,npr967652562138_20100930,npr967652562138,20100930,1,C81,C81.9,0,2
2,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,npr967652562138_20110527,npr967652562138,20110527,1,C81,C81.9,0,2
3,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,npr967652562138_20111005,npr967652562138,20111005,1,C81,C81.9,0,2
4,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,npr967652562138_20120206,npr967652562138,20120206,1,C81,C81.9,0,2


In [5]:
df = df.applymap(lambda x: str(x).replace('T7 Shield1', 'T7 Shield'))

In [6]:
df.head()

,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,unique_patient_ID_scan_date,patient_ID,scan_date,sex,diagnosis_groups,diagnosis_groups_new,GT_diagnosis_label,GT_diagnosis_label_new
0,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,npr967652562138_20100730,npr967652562138,20100730,1,C81,C81.9,0,2
1,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,npr967652562138_20100930,npr967652562138,20100930,1,C81,C81.9,0,2
2,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,npr967652562138_20110527,npr967652562138,20110527,1,C81,C81.9,0,2
3,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,npr967652562138_20111005,npr967652562138,20111005,1,C81,C81.9,0,2
4,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,npr967652562138_20120206,npr967652562138,20120206,1,C81,C81.9,0,2


In [6]:
df.to_excel("/media/andres/T7 Shield/UCAN_project/Dataframes/Generated/dataset_for_model_classification_training_new_diagnosis.xlsx", index=False)

In [1]:
k_fold = 10
outcome = 'GT_diagnosis_label'
def stratified_split(df_clean, k, outcome):
    df_list=[ df_clean[df_clean[outcome]==x].reset_index(drop=True) for x in range(3) ]
    factor_list= [ round(x.shape[0]/k_fold) for x in df_list ]

    if k == (k_fold - 1):
        patients_for_val = []
        for x,f in zip(df_list,factor_list):
            patients_for_val.extend(x[f*k:].unique_patient_ID_scan_date.tolist())
        df_val = df_clean[df_clean.unique_patient_ID_scan_date.isin(patients_for_val)].reset_index(drop=True)

    else:
        patients_for_val = []
        for x,f in zip(df_list,factor_list):
            patients_for_val.extend(x[f*k:f*k+f].unique_patient_ID_scan_date.tolist())
        df_val = df_clean[df_clean.unique_patient_ID_scan_date.isin(patients_for_val)].reset_index(drop=True)

    df_train = df_clean[~df_clean.unique_patient_ID_scan_date.isin(patients_for_val)].reset_index(drop=True)

    return df_train, df_val

In [10]:
import sys
import os
import pandas as pd

In [11]:
parent_dir = os.path.abspath('../')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from Utils import utils

In [13]:
config = utils.read_config()
system = 1 # 1 or 2
if system == 1:
    PATH = config["Source"]["paths"]["source_path_system_1"]
elif system == 2:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    PATH = config["Source"]["paths"]["source_path_system_2"]
else:
    PATH = ""

df_path = PATH + config["collages_for_classification_dataframe"]
#df_path = PATH + config["collages_for_classification_dataframe_new_diagnosis"]

df = pd.read_excel(df_path)
df_sorted = df.sort_values(by="scan_date")
df_sorted['scan_date'] = df_sorted['scan_date'].astype(str)
df_sorted['unique_patient_ID_scan_date'] = df_sorted['patient_ID'] + '_' + df_sorted['scan_date']
df_sorted = df_sorted.drop(columns=['patient_ID', 'scan_date'])

#   df = df.sort_values(by="unique_patient_ID_scan_date")

In [16]:
df_sorted.head()

,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,sex,diagnosis_groups,GT_diagnosis_label,unique_patient_ID_scan_date
699,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,1,C81,0,npr967652562138_20100730
244,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,1,C83,1,npr355725597861_20100816
600,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,0,C85,2,npr824033832482_20100902
235,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,1,C81,0,npr351786103610_20100913
700,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,/media/andres/T7 Shield/UCAN_project/collages/...,1,C81,0,npr967652562138_20100930


In [19]:
for k in range(k_fold):
     df_train, df_val = stratified_split(df_sorted, k, outcome)
     print(df_train.shape, df_val.shape)

(648, 14) (72, 14)
(648, 14) (72, 14)
(648, 14) (72, 14)
(648, 14) (72, 14)
(648, 14) (72, 14)
(648, 14) (72, 14)
(648, 14) (72, 14)
(648, 14) (72, 14)
(648, 14) (72, 14)
(648, 14) (72, 14)


In [18]:
print(df_train.shape)

(648, 14)
